<a href="https://colab.research.google.com/gist/addvennture/9ded050983451438740a24932069bf46/aconex_mail_data_query_rev0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# (0) INTRODUCTION

This notebook simplifies the process of querrying Aconex web services to retrieve project mail data across a portfolio of projects for a single user. Specifically, it retrieves all projects for a user using the Projects API and passes these to the Mail API to retrieve up to 250 projects mails per day per project and stores them either locally in a temporary csv file or to a user specified google drive account. By default up to 11 mail field/value pairs will be retrieved. Users can set the date range for the search window to limit the size of the data request/runtime, for larger projects / portfolios a shorter search window is suggested. 

Please see the Readme file for set up details and usage tips. 

# (1) IMPORTING REFERENCED LIBRARIES

In [ ]:
#Importing Python Libraries
import requests as rq
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import csv

# Import PyDrive and associated libraries.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# (2) SETTING SEARCH DATE RANGE

In [ ]:
#@title Enter Search Dates { display-mode: "form" }
strt_date = "2021-02-01" #@param {type:"date"}
end_date = "2021-02-02" #@param {type:"date"}
#Setting search window dates
#strt_date='20201207' #start date for search
#end_date='20201209' #end date for search 
srch_wndw=pd.date_range(strt_date, end_date)
#print(srch_wndw[0])

dates=[]
for day in srch_wndw:
  a=str(day)[0:10]
  b=a.replace('-','')
  dates.append(b)
#print(dates) #use to check date range

# (3) PLACEHOLDER FOR FUTURE GOOGLE DRIVE CONNECTION

In [ ]:
# THIS IS AN OPTIONAL STEP TO SAVE OUTPUTS TO A GOOGLE DRIVE ACCOUNT, IF NOT USED FILES WILL BE DELETED WHEN A SESSION IS CLOSED
# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# (4) SETTING UP THE ACONEX API CONNECTION

In [ ]:
# ENTER API CREDENTIALS
#@title Aconex API Authorisation Credentials
username = "" #@param {type:"string"}
password = "" #@param {type:"string"}
key = "" #@param {type:"string"}
b_url='https://uk1.aconex.co.uk/api/' #base path for all Aconex web services
headers={'Accept': 'application/vnd.aconex.mail.v2+xml','Authorization': 'Bearer {0}'.format(key)}

# (5) DEFINED FUNCTIIONS

In [ ]:
#Generates a list of project mail, using "List Mail (Version 2) in the Aconex Mail API"
def get_inmail(aconex_id,strt_date,end_date):
    mailbox='inbox' #mailbox for querrying, available alternatives are: inbox, sentbox, draftbox
    srch_qry='sentdate:[{0} TO {1}]'.format(strt_date,end_date)
    r_fields=('corrtypeid') #check mail schema first to confirm result field availability - Available options: subject,sentdate,responsedate
    s_field='sentdate' #results sorted by correspondence type 
    data=rq.get('https://uk1.aconex.co.uk/api/projects/{0}/mail'.format(aconex_id),auth=(username,password),headers=headers,params=({'mail_box':mailbox,'search_query':srch_qry,'return_fields':r_fields,'sort_field':s_field}))
    root=ET.fromstring(data.text)
    return root

In [ ]:
#Generates a list of project mail, using "List Mail (Version 2) in the Aconex Mail API"
def get_outmail(aconex_id,strt_date,end_date):
    mailbox='sentbox' #mailbox for querrying, available alternatives are: inbox, sentbox, draftbox
    srch_qry='sentdate:[{0} TO {1}]'.format(strt_date,end_date)
    r_fields=('corrtypeid') #check mail schema first to confirm result field availability - Available options: subject,sentdate,responsedate
    s_field='sentdate' #results sorted by correspondence type 
    data=rq.get('https://uk1.aconex.co.uk/api/projects/{0}/mail'.format(aconex_id),auth=(username,password),headers=headers,params=({'mail_box':mailbox,'search_query':srch_qry,'return_fields':r_fields,'sort_field':s_field}))
    root=ET.fromstring(data.text)
    return root

In [ ]:
#GETS metadata for individual mails using 'View Mail Metadata (V2)'
def get_mailmeta(aconex_id,mail_id):
    data=rq.get('https://uk1.aconex.co.uk/api/projects/{0}/mail/{1}'.format(aconex_id,mail_id),auth=(username,password),headers=headers)
    #root=ET.fromstring(data.text)
    return data

In [ ]:
#Function creates a dictionary of mail ids for a project
def prj_mails(mail_root):
  prj_mails={}
  counter=0
  for mail in mail_root.findall('.//Mail'):
    prj_mails[counter]=mail.attrib
    counter=counter+1
  return prj_mails

In [ ]:
#THIS FUNCTION CAPTURES SOME OF THE AVAILABLE #Option to permanently save temporary csv file to Google Drive.
#Note requires authentication in Section 3.
def maildata3(mailmeta_root):
  try:
    prj_id=mailmeta_root.find('ProjectId').text
  except:
    prj_id='NONE'
  try:
    corrtype=mailmeta_root.find('CorrespondenceType').text
  except:
    corrtype='NONE'
  try:
    sentdate=mailmeta_root.find('SentDate').text
  except:
    sentdate='NONE'
  try:
    status=mailmeta_root.find('Status').text
  except:
    status='NONE'
  try:
    subject=mailmeta_root.find('Subject').text
  except:
    subject='NONE'
  try:
    mailthread=mailmeta_root.find('ThreadId')
  except:
    mailthread='NONE'

  mail=mailmeta_root.attrib['MailId']

  coredata=[prj_id,mail,sentdate,corrtype,subject,status,mailthread]

  for mff in mailmeta_root.findall('.//MailFormField'):
      coredata.append(mff.find('Label').text)
      coredata.append(mff.find('Value').text)
  return coredata

# (6) GETTING LIST OF PROJECTS

In [ ]:
#Running project search query and retrieving project list for user.
prj_headers={'User':username,'Authorization': 'Bearer {0}'.format(key)}
prj_data=rq.get('https://uk1.aconex.co.uk/api/projects',auth=(username,password),headers=prj_headers)
prj_root=ET.fromstring(prj_data.text) #Bringing data into element tree for processing
#[elem.tag for elem in prj_root.iter()] #Optional step to check data structure
#prj_root.attrib #Check on total number of projects 

In [ ]:
#Generating a list of project ids for mail search queries.
prj_ids=[]
for child in prj_root.iter('ProjectId'):
    prj_ids.append(child.text)

# (7) FETCHING & SAVING MAIL DATA

In [ ]:
#Opening temporary csv file to write data out to. 
with open('TEMP_DATA.csv',mode='w') as csvfile:
    csvwriter=csv.writer(csvfile)

In [ ]:
#Getting data for all INBOX mails on all projects for the search date range and writting them to a csv file
#Note depending on the size of the query this may take a while to run.
for day in dates:
  for prj_id in prj_ids:
    mails=get_inmail(prj_id,day,day) 
    p_mails=prj_mails(mails)
    #print(p_mails)
    counter=0
    for mail in p_mails:
      mail_id=p_mails[counter]['MailId']
      #print(prj_id,mail_id)
      metadata=get_mailmeta(prj_id,mail_id)
      mailmeta_root=ET.fromstring(metadata.text)
      md=maildata3(mailmeta_root)
      df=pd.DataFrame(md)
      dft=df.transpose()
      dft.to_csv('TEMP_DATA.csv',header=False,mode='a')
      counter=counter+1
      #print(md)

In [ ]:
#Getting data for all SENT mails on all projects for the search date range and adding them to the AGM_data.csv file
#Note depending on the size of the query this may take a while to run.
for day in dates:
  for prj_id in prj_ids:
    mails=get_outmail(prj_id,day,day) 
    p_mails=prj_mails(mails)
    #print(p_mails)
    counter=0
    for mail in p_mails:
      mail_id=p_mails[counter]['MailId']
      #print(prj_id,mail_id)
      metadata=get_mailmeta(prj_id,mail_id)
      mailmeta_root=ET.fromstring(metadata.text)
      md=maildata3(mailmeta_root)
      df=pd.DataFrame(md)
      dft=df.transpose()
      dft.to_csv('TEMP_DATA.csv',header=False,mode='a')
      counter=counter+1
      #print(md)

In [ ]:
#Option to permanently save temporary csv file to Google Drive.
#Note requires authentication in Section 3.
ftd=drive.CreateFile({'title':'Aconex_Mail_Data{0}_{1}'.format(strt_date,end_date)}) #Creates file with a name matching the search window dates
ftd.SetContentFile('/content/TEMP_DATA.csv') #sets processed mail data from above cells as content to upload to new file
ftd.Upload() #uploads content 
print('title:%s, id:%s' % (ftd['title'],ftd['id'])) #prints file name and google drive id